# Reactome Pathway Extraction

To impose the penalty for crossing Reactome Pathways, we extract those pathways here and subset the graph accordingly.

In [1]:
import pandas as pd
import requests
import numpy as np
import os.path as osp
import itertools

First, let's get data from Reactome:

In [2]:
# write code which gets the file from a raw github link
# and separates each line by whitespace
# and then creates a dictionary where the key is the first item, and the values are a list of the rest of the items


def get_file_from_github(url):
    """
    Gets a file from a raw github link
    """
    
    r = requests.get(url)
    return r.text

In [3]:
url = 'https://raw.githubusercontent.com/pathwayforte/pathway-forte/master/data/gmt_files/reactome.gmt'

r = requests.get(url)
# get the text from the response in a tab-delimited format

content = r.text

lines = content.split('\n')  # Split by newline
data = [line.split('\t') for line in lines]

The first tow items of the list are the pathway ID and the database (reactome). The rest of the items are the genes in the pathway. We'll just get those.

In [4]:
parsed_data = dict()
for d in data:
    if len(d) > 2:
        parsed_data[d[0]] = d[2::]

We'll take the four longest lists (the top 4 biggest pathways/subgraphs)

In [5]:
# Sort dictionary items by list length in descending order
sorted_data = sorted(parsed_data.items(), key=lambda item: len(item[1]), reverse=True)

# Get the longest 5 lists
longest_lists = dict(sorted_data[:4])

In [6]:
longest_lists.keys()

dict_keys(['R-HSA-162582', 'R-HSA-168256', 'R-HSA-1430728', 'R-HSA-392499'])

These are:

- R-HSA-162582 : Signal Transduction
- R-HSA-168256: Immune System
- R-HSA-1430728: Metabolism
- R-HSA-392499: Metabolism of proteins

Let's see how many of these proteins are in the KG we made:

In [7]:
proteins_in_pathways = {prot for lst in longest_lists.values() for prot in lst}

In [8]:
print(f"{len(proteins_in_pathways)} unique proteins in these pathways.")

7125 unique proteins in these pathways.


## Protein / gene ID Mapping

Let's map these to NCBI gene IDs based on a download from the [HGNC download page](https://www.genenames.org/).

In [9]:
mapping_file = 's3://enveda-datascience/lauren/gene_symbol_to_ncbi.txt'

name2ncbi = pd.read_csv(
    mapping_file, 
    sep='\t',
    usecols=['Approved symbol', 'NCBI Gene ID(supplied by NCBI)'],
    dtype=str,
)
name2ncbi.dropna(subset=['Approved symbol'], inplace=True)
name2ncbi.head(2)

,Approved symbol,NCBI Gene ID(supplied by NCBI)
0,A1BG,1
1,A1BG-AS1,503538


In [10]:
name2ncbi = {row['Approved symbol']: row['NCBI Gene ID(supplied by NCBI)'] for i, row in name2ncbi.iterrows()}

In [11]:
ncbigenes_in_pathways = {f'ncbigene:{name2ncbi[p]}' for p in proteins_in_pathways if p in name2ncbi}

In [12]:
print(f"{len(ncbigenes_in_pathways)} of those mapped to NCBI IDs.")

7096 of those mapped to NCBI IDs.


Now load in the KG and get all of those proteins:

In [13]:
KG_DIR = '../data/kg'

In [14]:
kg = pd.read_csv(osp.join(KG_DIR, 'final_kg.tsv'), sep='\t')

In [15]:
kg.head()

,source,source_node_type,target,target_node_type,edge_type
0,pubchem.compound:10607,Compound,ncbigene:3553,Gene,upregulates
1,pubchem.compound:10607,Compound,ncbigene:203068,Gene,downregulates
2,pubchem.compound:10607,Compound,ncbigene:54658,Gene,downregulates
3,pubchem.compound:10607,Compound,ncbigene:7153,Gene,downregulates
4,pubchem.compound:10607,Compound,ncbigene:7277,Gene,downregulates


In [16]:
target_sets = set(kg[kg['target'].str.startswith('ncbigene')]['target'].tolist())
source_sets = set(kg[kg['source'].str.startswith('ncbigene')]['source'].tolist())
proteins_in_kg = target_sets.union(source_sets)
len(proteins_in_kg)

9301

In [17]:
len(ncbigenes_in_pathways & proteins_in_kg)

5462

In [18]:
print(f'{len(ncbigenes_in_pathways & proteins_in_kg)/len(proteins_in_kg)*100}% of proteins are covered by these subgraphs.')

58.72486829373186% of proteins are covered by these subgraphs.


How much of an overlap do we have between subgraphs / pathways?

In [19]:
don = set()
overlap_dict = dict()

for key, val in longest_lists.items():
    for key2, val2 in longest_lists.items():
        if key == key2 or (key, key2) in don or (key2, key) in don:
            continue
        don.add((key, key2))
        don.add((key2, key))
        overlap = len((set(val) & set(val2))) / len(set(val).union(set(val2)))
        overlap_dict[(key, key2)] = overlap
        print(f'{overlap*100}% overlap between {key} and {key2}')

16.295781177561427% overlap between R-HSA-162582 and R-HSA-168256
6.351263771872976% overlap between R-HSA-162582 and R-HSA-1430728
9.372156505914468% overlap between R-HSA-162582 and R-HSA-392499
6.029411764705882% overlap between R-HSA-168256 and R-HSA-1430728
12.925717350496111% overlap between R-HSA-168256 and R-HSA-392499
8.62796833773087% overlap between R-HSA-1430728 and R-HSA-392499


## Subset the KG

Next, let's subset our KG and see how many of our drug-BP labels we have left.

In [20]:
filtered_kg = kg.loc[(kg['source_node_type'] != 'Gene') | (kg['source'].isin(ncbigenes_in_pathways))]
filtered_kg = filtered_kg.loc[(filtered_kg['target_node_type'] != 'Gene') | (filtered_kg['target'].isin(ncbigenes_in_pathways))]

In [21]:
print(f"Dropped {len(kg) - len(filtered_kg)} edges.")
print(f"That's {(len(kg) - len(filtered_kg)) / len(kg) * 100}% of the original graph.")

Dropped 28329 edges.
That's 29.2727535752666% of the original graph.


Have we lost any drug-BP edges?

In [23]:
len(filtered_kg.loc[filtered_kg['edge_type'] == 'induces'])

1803

In [24]:
len(kg.loc[kg['edge_type'] == 'induces'])

1803

We actually have all of our drug-BP pairs left. Are all of the proteins connected to these drugs and BPs actually connected to the rest of the network still?

In [25]:
# get the proteins connected to other proteins
prots_connected = set()
for i, row in filtered_kg.loc[filtered_kg['edge_type'] == 'interacts'].iterrows():
    prots_connected.add(row['source'])
    prots_connected.add(row['target'])

In [26]:
# get the BP nodes connected to a protein, which is connected to other proteins
bp_nodes = {row['target'] for i, row in filtered_kg.loc[filtered_kg['edge_type'] == 'participates'].iterrows() if row['source'] in prots_connected}
# get the drug nodes connected to a protein, which is connected to other proteins
drug_nodes = {row['source'] for i, row in filtered_kg.loc[filtered_kg['edge_type'].isin({'downregulates', 'upregulates'})].iterrows() if row['target'] in prots_connected}

In [27]:
drugbp_pairs = filtered_kg.loc[filtered_kg['edge_type'] == 'induces']

In [28]:
len(drugbp_pairs)

1803

In [41]:
drug_bp_pairs = drugbp_pairs.loc[(drugbp_pairs['source'].isin(drug_nodes)) & (drugbp_pairs['target'].isin(bp_nodes))]
len(drug_bp_pairs)

1705

What about the validation set?

In [32]:
val = pd.read_csv(osp.join(KG_DIR, 'test.tsv'), sep='\t', header=0)

In [35]:
len(val)

48

In [42]:
dm_db_pairs = val.loc[(val['source'].isin(drug_nodes)) & (val['target'].isin(bp_nodes))]
len(dm_db_pairs)

37

It seems like we lose some of our labels, but we still have plenty. Let's make new datasets out of this.

## New Splits for Subgraph Penalty

In [38]:
SPLITS_DIR = osp.join(KG_DIR, 'splits')
SG_DIR = osp.join(SPLITS_DIR, 'subgraph_penalty')

In [43]:
total_positives = len(drug_bp_pairs) + len(dm_db_pairs)
print(f"After subsetting, we have {total_positives} connected positive pairs left.")

After subsetting, we have 1742 connected positive pairs left.


We want to get the same proportions as the previous split:

In [44]:
proportions = round(0.6 * total_positives), round(0.2 * total_positives), round(0.2 * total_positives) - len(dm_db_pairs)
proportions

(1045, 348, 311)

In [45]:
# write a function which separates the dataframe into train, val and test sets of defined sizes
def train_test_split(df, train_size, val_size, test_size):
    df = df.sample(frac=1, random_state=7).reset_index(drop=True)
    train = df[:train_size]
    val = df[train_size:train_size+val_size]
    test = df[train_size+val_size:train_size+val_size+test_size]
    return train, val, test

In [46]:
train, val, test = train_test_split(drug_bp_pairs, proportions[0], proportions[1], proportions[2])

Check to make sure it worked:

In [47]:
len(train), len(val), len(test)

(1045, 348, 311)

Ensure there is no overlap between sets:

In [48]:
train_pairs = {(row['source'], row['target']) for i, row in train.iterrows()}
test_pairs = {(row['source'], row['target']) for i, row in test.iterrows()}
val_pairs = {(row['source'], row['target']) for i, row in val.iterrows()}

print(train_pairs & test_pairs)
print(train_pairs & val_pairs)
print(test_pairs & val_pairs)

set()
set()
set()


Now, combine the DrugMechDB pairs with the test set:

In [49]:
test = pd.concat([test, dm_db_pairs]).sample(frac=1, random_state=7).reset_index(drop=True)
test

,source,source_node_type,target,target_node_type,edge_type
0,pubchem.compound:10631,Compound,GO:0061024,Biological Process,induces
1,pubchem.compound:6167,Compound,GO:0030705,Biological Process,induces
2,pubchem.compound:3059,Compound,GO:0007165,Biological Process,induces
3,pubchem.compound:16197727,Compound,GO:0042438,Biological Process,induces
4,pubchem.compound:5743,Compound,GO:0006954,Biological Process,induces
...,...,...,...,...,...
343,pubchem.compound:6167,Compound,GO:0050877,Biological Process,induces
344,pubchem.compound:44462760,Compound,GO:0031638,Biological Process,induces
345,pubchem.compound:941361,Compound,GO:0048856,Biological Process,induces
346,pubchem.compound:8378,Compound,GO:0042592,Biological Process,induces


Take the test and validation edges out the KG:

In [50]:
kg = kg.loc[kg['edge_type'] != 'induces']
kg_polo = pd.concat([kg, train]).sample(frac=1, random_state=7).reset_index(drop=True)

The number of `induces` edges in the KG should be the same as the number in the training set:

In [51]:
len(kg_polo.loc[kg_polo['edge_type'] == 'induces']) == len(train)

True

Write everything to files:

In [52]:
kg.to_csv(osp.join(SG_DIR, 'kg_no_cmp_bp.tsv'), sep='\t', index=False)
kg_polo.to_csv(osp.join(SG_DIR, 'kg_with_train_smpls.tsv'), sep='\t', index=False)

train.to_csv(osp.join(SG_DIR, 'train.tsv'), sep='\t', index=False)
val.to_csv(osp.join(SG_DIR, 'dev.tsv'), sep='\t', index=False)
test.to_csv(osp.join(SG_DIR, 'test.tsv'), sep='\t', index=False)

## Subgraph Map

Finally, we need a file that tells us the points at which the model should impose a penalty for subgraph crosses.

In [ ]:
# TODO